## NextBus: Stop-Level Bus Tracking with Distance and ETA Estimation

### Download bus stop and route BMTC data using SDK from GDI platform

catalogue link: https://catalogue.geospatial.org.in/dataset/a8691b7a-6fd6-49e6-bafb-322773ca12e9

Below script runs the command line to fetch vector data using GDI SDK
1. The client credentials (id and secret) are made avialable for download at the registering as user of GDI platform. In case credentials are lost, go to your profile, reset the credentials and download for further use.
2. For each data of interest, 'dataset_id' can be copied from the catalogue page
3. In case file_name is specified without file extension (.geojson) the code will automatically add file extension and save in 'Data' folder



In [ ]:
'''
Install gdi python SDK using below pip statement
pip install git+https://github.com/datakaveri/gdi-python-sdk.git
-------------------------------------------------------------------------------
Command line:
gdi get-vector-data --client-id <client_id> --client-secret <client_secret> --role consumer --resource-id <dataset_id> --store-artifact local --config-path 'config.json' --file-path <file_name>
'''

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
import os
import time

# ----------------------------------------
# Create interactive widgets for user input
# ----------------------------------------

# Widget for client ID (text input)
client_id_widget = widgets.Text(
    description="Client ID:",
    placeholder="Enter Client ID",
    layout=widgets.Layout(width='90%')
)

# Widget for client secret (hidden input)
client_secret_widget = widgets.Password(
    description="Client Secret:",
    placeholder="Enter Client Secret",
    layout=widgets.Layout(width='90%')
)

# Widget for resource ID (text input)
resource_id_widget = widgets.Text(
    description="Resource ID:",
    placeholder="Enter Resource ID",
    layout=widgets.Layout(width='90%')
)

# Widget for output filename (text input)
filename_widget = widgets.Text(
    description="Filename:",
    placeholder="e.g. Road_Varanasi",
    layout=widgets.Layout(width='90%')
)

# Button to execute the command
execute_button = widgets.Button(
    description="Fetch Resource",
    button_style="success"   # Makes the button green
)

# Output area to display logs and messages
output = widgets.Output()

# ------------------------------------------------
# Debounce logic to prevent multiple rapid clicks
# ------------------------------------------------

last_click_time = 0

def on_click(b):
    """
    Handler function triggered when the button is clicked.
    It reads the widget values and runs the GDI SDK command.
    """
    global last_click_time

    # Debounce: ignore clicks occurring within 0.5 seconds
    current_time = time.time()
    if current_time - last_click_time < 0.5:
        return
    last_click_time = current_time

    with output:
        clear_output()

        # Read user inputs from widgets
        client_id = client_id_widget.value.strip()
        client_secret = client_secret_widget.value.strip()
        resource_id = resource_id_widget.value.strip()
        filename = filename_widget.value.strip()

        # Ensure the output filename has a .geojson extension
        if not filename.endswith(".geojson"):
            filename += ".geojson"

        # Create output folder called 'Data' if it doesn't exist
        output_dir = "Data"
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, filename)

        # Prepare the GDI SDK command to fetch vector data
        cmd = [
            "gdi", "get-vector-data",
            "--client-id", client_id,
            "--client-secret", client_secret,
            "--role", "consumer",
            "--resource-id", resource_id,
            "--store-artifact", "local",
            "--config-path", "config.json",
            "--file-path", output_path
        ]

        # # Print the command for debugging purposes
        # print("Executing command:")
        # print(" ".join(cmd))
        # print()

        try:
            # Run the command and capture output
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)

            # Print success message and output file path
            # print("[DONE] Collection download complete.")
            print(f"GDI vector resource successfully downloaded to:\n{output_path}")

        except subprocess.CalledProcessError as e:
            # Print error message if command fails
            print("[ERROR] Failed to fetch resource.")
            print(e.stderr)

# ------------------------------------------------
# Attach the click handler to the button
# ------------------------------------------------

execute_button.on_click(on_click)

# ------------------------------------------------
# Display all widgets and the output area
# ------------------------------------------------

display(
    client_id_widget,
    client_secret_widget,
    resource_id_widget,
    filename_widget,
    execute_button,
    output
)


Text(value='', description='Client ID:', layout=Layout(width='90%'), placeholder='Enter Client ID')

Password(description='Client Secret:', layout=Layout(width='90%'), placeholder='Enter Client Secret')

Text(value='', description='Resource ID:', layout=Layout(width='90%'), placeholder='Enter Resource ID')

Text(value='', description='Filename:', layout=Layout(width='90%'), placeholder='e.g. Road_Varanasi')

Button(button_style='success', description='Fetch Resource', style=ButtonStyle())

Output()

NOTE:
- Replace the file name wherever necessary throughout the code blocks.  
- The notebook assumes the data are saved as **bmtc_route** and **bmtc_stop** in GeoJSON format.

### Explore downloaded stop and route data its attributes

In [2]:
import os
import fiona
import geopandas as gpd

# Set the folder path containing the GeoJSON files
folder_path = "./Data"

# Iterate through the files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.geojson'):
        file_path = os.path.join(folder_path, file_name)
        print(f"\n File: '{file_name}'")

        try:
            with fiona.Env():
                # Load the GeoJSON into a GeoDataFrame
                gdf = gpd.read_file(file_path)
                
                # Show preview
                print("\n Sample records:")
                print(gdf.head())

                # Print attribute names and their data types
                print("\nAttributes and Data Types:")
                for attr, dtype in gdf.dtypes.items():
                    print(f"- {attr}: {dtype}")

        except Exception as e:
            print(f"Error reading {file_name}: {e}")



 File: 'bmtc_route.geojson'

 Sample records:
              shape_id        route_id  \
0             104 DOWN             104   
1               104 UP             104   
2  107-C D14G-SBS DOWN  107-C D14G-SBS   
3    107-C D14G-SBS UP  107-C D14G-SBS   
4     111 CMS-KBR DOWN     111 CMS-KBR   

                                     trip_id  \
0  39650,39651,39652,39653,39654,39655,39656   
1  39643,39644,39645,39646,39647,39648,39649   
2                          39565,39566,39567   
3                          39562,39563,39564   
4                                44387,44388   

                                route_long_name route_short_name  agency_id  \
0      Kempegowda Bus Station ⇔ Sadashivanagara              104          1   
1      Kempegowda Bus Station ⇔ Sadashivanagara              104          1   
2     Depot-14 Gate ⇔ Shivajinagara Bus Station            107-C          1   
3     Depot-14 Gate ⇔ Shivajinagara Bus Station            107-C          1   
4  Cubbon Park M

### Update route file with new_route_id based on lookup file (Live_feed_route_id_lookup.csv) in Data folder 

In [3]:
import geopandas as gpd
import pandas as pd
import os

# === File Paths ===
geojson_path = "./Data/bmtc_route.geojson"
csv_path = "./Data/Live_feed_route_id_lookup.csv"
output_path = "./Data/bmtc_route_updated.gpkg"

# === Read files ===
gdf = gpd.read_file(geojson_path)
df_lookup = pd.read_csv(csv_path)

# === Normalize for safe comparison ===
gdf["route_id"] = gdf["route_id"].astype(str).str.strip().str.lower()
df_lookup["route_short_name"] = df_lookup["route_short_name"].astype(str).str.strip().str.lower()
df_lookup["route_long_name"] = df_lookup["route_long_name"].astype(str).str.strip().str.lower()
df_lookup["route_id"] = df_lookup["route_id"].astype(str).str.strip()

# === Build mapping from short/long names to route_id ===
short_map = df_lookup.set_index("route_short_name")["route_id"].to_dict()
long_map = df_lookup.set_index("route_long_name")["route_id"].to_dict()

# === Function to find matching live_feed route_id ===
def match_route_id(rid):
    return short_map.get(rid) or long_map.get(rid)

# === Add new column ===
gdf["new_route_id"] = gdf["route_id"].apply(match_route_id)

# === Save output ===
os.makedirs("Data", exist_ok=True)
gdf.to_file(output_path, driver="GPKG")
print(f"Saved updated GeoPackage to: {output_path}")


Saved updated GeoPackage to: ./Data/bmtc_route_updated.gpkg


In [ ]:
# gdf

,shape_id,route_id,trip_id,route_long_name,route_short_name,agency_id,route_type,stop_list,geometry,new_route_id
0,104 DOWN,104,"39650,39651,39652,39653,39654,39655,39656",Kempegowda Bus Station ⇔ Sadashivanagara,104,1,3,"28095,39297,20573,20564,20562,20589,20989,2097...","LINESTRING (77.58027 13.00865, 77.58017 13.008...",8630
1,104 UP,104,"39643,39644,39645,39646,39647,39648,39649",Kempegowda Bus Station ⇔ Sadashivanagara,104,1,3,"20921,20603,35303,20702,20988,20590,20561,2057...","LINESTRING (77.57141 12.97751, 77.57098 12.977...",8630
2,107-C D14G-SBS DOWN,107-c d14g-sbs,"39565,39566,39567",Depot-14 Gate ⇔ Shivajinagara Bus Station,107-C,1,3,"21172,22068,21869,22585,24018,24019,24020,2334...","LINESTRING (77.60323 12.98396, 77.60326 12.984...",8548
3,107-C D14G-SBS UP,107-c d14g-sbs,"39562,39563,39564",Depot-14 Gate ⇔ Shivajinagara Bus Station,107-C,1,3,"36584,23786,23787,23788,23789,24172,23790,2379...","LINESTRING (77.59372 13.02851, 77.59357 13.028...",8548
4,111 CMS-KBR DOWN,111 cms-kbr,"44387,44388",Cubbon Park Metro Station ⇔ Kaval Byrasandra,111 CMS-KBR,1,3,"20918,21073,21201,20760,20834,23786,23787,2378...","LINESTRING (77.60769 13.01974, 77.60752 13.019...",11844
...,...,...,...,...,...,...,...,...,...,...
6962,YTTMC-GGP-PTCA UP,yttmc-ggp-ptca,39869,Yeshawanthapura Bus Station ⇔ Platinum City Ap...,YTTMC-GGP-PTCA,1,3,"21288,20791,21131,21016,20790,20723,27701","LINESTRING (77.55662 13.01801, 77.55658 13.017...",8947
6963,YTTMC-VYKVLP DOWN,yttmc-vykvlp,39878,Yeshawanthapura Bus Station ⇔ Vaiyalikaval Pol...,YTTMC-VYKVLP,1,3,"21245,31919,29391,21213,21288","LINESTRING (77.57849 13.00183, 77.57853 13.002...",8957
6964,YTTMC-VYKVLP UP,yttmc-vykvlp,39877,Yeshawanthapura Bus Station ⇔ Vaiyalikaval Pol...,YTTMC-VYKVLP,1,3,"24016,21214,35603,29786,21244","LINESTRING (77.55657 13.01771, 77.55671 13.018...",8957
6965,YTTMC-YPRS DOWN,yttmc-yprs,44403,Yeshawanthapura Bus Station ⇔ Yeshawanthapura ...,YTTMC-YPRS,1,3,"27453,20792,21288","LINESTRING (77.55066 13.02349, 77.55082 13.023...",11852


### Update the stop file with old and new route id list and save as "bus_stop_with_routes.gpkg" in Data folder

In [4]:
import geopandas as gpd
from tqdm import tqdm

# --- Step 1: Load GeoPackage and GeoJSON files ---
routes_path = "./Data/bmtc_route_updated.gpkg"
stops_path = "./Data/bmtc_stop.geojson"
routes_gdf = gpd.read_file(routes_path)
stops_gdf = gpd.read_file(stops_path)

# --- Step 2: Build mapping from stop_id → list of shape_ids and shape_id → new_route_id ---
stop_to_shape_ids = {}
shape_to_new_route_id = routes_gdf.set_index("shape_id")["new_route_id"].to_dict()

# Build stop_id → shape_id mapping
for _, row in tqdm(routes_gdf.iterrows(), total=len(routes_gdf), desc="Building stop → shape_id mapping"):
    shape_id = row["shape_id"]
    stop_list_str = row.get("stop_list", "")

    if not stop_list_str or not isinstance(stop_list_str, str):
        continue  # Skip missing or invalid stop_list

    stop_list = [s.strip() for s in stop_list_str.split(",") if s.strip()]
    for stop_id in stop_list:
        stop_to_shape_ids.setdefault(stop_id, []).append(shape_id)

# --- Step 3: Add 'route_list' and 'new_route_ids' columns to stops_gdf ---
def get_shape_ids(stop_id):
    return stop_to_shape_ids.get(str(stop_id), [])

def get_new_route_ids(stop_id):
    shape_ids = get_shape_ids(stop_id)
    route_ids = [shape_to_new_route_id.get(sid) for sid in shape_ids if sid in shape_to_new_route_id]
    return list(filter(None, route_ids))  # remove None values

stops_gdf["route_list"] = stops_gdf["stop_id"].astype(str).apply(get_shape_ids)
stops_gdf["new_route_ids"] = stops_gdf["stop_id"].astype(str).apply(get_new_route_ids)

# Convert lists to comma-separated strings
stops_gdf["route_list"] = stops_gdf["route_list"].apply(lambda x: ",".join(map(str, x)) if x else "")
stops_gdf["new_route_ids"] = stops_gdf["new_route_ids"].apply(lambda x: ",".join(map(str, x)) if x else "")

# --- Step 4: Save updated stops_gdf to GPKG ---
output_path = "./Data/bus_stop_with_routes.gpkg"
stops_gdf.to_file(output_path, driver="GPKG")
print(f"Saved to: {output_path}")



Building stop → shape_id mapping: 100%|██████████| 6967/6967 [00:00<00:00, 8848.03it/s]


Saved to: ./Data/bus_stop_with_routes.gpkg


In [ ]:
# stops_gdf

,stop_name,zone_id,stop_id,stop_lat,stop_lon,geometry,route_list,new_route_ids
0,(Theneyuru)I Basapura Gate,26089,26089,13.22602,77.83624,POINT (77.83624 13.22602),381-C UP,6025
1,10th Cross Lingadhiranahalli,29374,29374,13.00951,77.47431,POINT (77.47431 13.00951),"243-L D31-LDH UP,Vivek_Test14 DOWN,Vivek_Test1...","11232,15294,15294"
2,10th Cross Magadi Road,20558,20558,12.97556,77.55562,POINT (77.55562 12.97556),"235-K UP,237-C UP,238-C UP,238-U KBS-AIT UP,23...","2207,2562,2566,10719,11223,13850,11227,10671,1..."
3,10th Cross Magadi Road,20559,20559,12.97574,77.55706,POINT (77.55706 12.97574),"235-K DOWN,237-C DOWN,238-C DOWN,238-U DOWN,23...","2207,2562,2566,10629,10719,11223,13850,10671,1..."
4,10th Stone NICE Road,29375,29375,12.87273,77.49860,POINT (77.4986 12.87273),"NICE-10 ELC-D43G DOWN,NICE-10 UP,NICE-10E DSP-...","14776,12584,15140,15121,6793,14802,15137,4924,..."
...,...,...,...,...,...,...,...,...
9257,Yettu Kodi,22840,22840,12.92116,77.85567,POINT (77.85567 12.92116),"304-M UP,304-P UP,304-Q D24G-GNLU UP,304-Q UP,...","4400,4376,5378,4385,5332,4364,4407,5352,4412,4..."
9258,Yettu Kodi,22841,22841,12.92118,77.85556,POINT (77.85556 12.92118),"304-M DOWN,304-P DOWN,304-Q D24G-GNLU DOWN,304...","4400,4376,5378,4385,4364,5332,4407,5352,4412,4..."
9259,Yoga Gangothri Trust,38495,38495,12.80147,77.30072,POINT (77.30072 12.80147),226-YB UP,14841
9260,Yoga Gangotri Trust,22851,22851,12.80137,77.30082,POINT (77.30082 12.80137),226-YB DOWN,14841


### Download live feed data of BMTC from IUDX platform for last 30 seconds. 

- Installed sdk using below pip statement -> 
pip install git+https://github.com/datakaveri/iudx-python-sdk


In [6]:
# To install IUDX SDK and access IUDX resource
!pip install git+https://github.com/datakaveri/iudx-python-sdk

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/datakaveri/iudx-python-sdk to /tmp/pip-req-build-w_w9dw42
  Running command git clone --filter=blob:none --quiet https://github.com/datakaveri/iudx-python-sdk /tmp/pip-req-build-w_w9dw42
  Resolved https://github.com/datakaveri/iudx-python-sdk to commit 5e5c696a234260abce8600e84c04310081063d99
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for iudx: filename=iudx-2.0.1-py3-none-any.whl size=23753 sha256=fd2a3fff3171067f759d4bb0eccb92dcb3a248bb9143a3c4486944b06a3de38d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ljncwett/wheels/f1/fc/90/c7e7b5d2130969c9a93d7b14a9a0132ba12da264f4a79556ec
Successfully built iudx
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1

In [ ]:

'''
Command to download data:

iudx --entity 8a9c53c2-7291-454e-b173-2d9bfe893eb1 \
  --clientid=<YOUR_CLIENT_ID> \
  --secret=<YOUR_CLIENT_SECRET> \
  --start $(date --date='30 seconds ago' '+%Y-%m-%dT%H:%M:%S%:z') \
  --end $(date '+%Y-%m-%dT%H:%M:%S%:z') \
  --entity-type=resource \
  --role=consumer \
  --download=data \
  --type=csv


  Enter your client credentials for IUDX platform. (Credentials of IUDX is different from GDI)
  Note the data is private and will require access granted from provider

  '''

In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
import datetime
from zoneinfo import ZoneInfo  # Python 3.9+
import os
import shutil

# === Widgets ===
client_id_widget = widgets.Text(
    value='',
    placeholder='Enter your client ID',
    description='Client ID:',
    disabled=False
)

secret_widget = widgets.Password(
    value='',
    placeholder='Enter your secret',
    description='Secret:',
    disabled=False
)

download_button = widgets.Button(
    description='Download Last 30s CSV',
    button_style='success'
)

output = widgets.Output()

# === Time Formatter ===
def format_iudx(dt: datetime.datetime) -> str:
    """Format datetime as YYYY-MM-DDTHH:MM:SS+05:30 (with colon in offset)."""
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=ZoneInfo('Asia/Kolkata'))
    s = dt.strftime('%Y-%m-%dT%H:%M:%S%z')
    return s[:-2] + ':' + s[-2:]

# === Download Function ===
def download_data(b):
    with output:
        clear_output()
        client_id = client_id_widget.value.strip()
        secret = secret_widget.value.strip()
        if not client_id or not secret:
            print("Please enter both Client ID and Secret.")
            return

        # Prepare time window
        now_ist = datetime.datetime.now(ZoneInfo('Asia/Kolkata'))
        start_ist = now_ist - datetime.timedelta(seconds=30)
        start_time = format_iudx(start_ist)
        end_time   = format_iudx(now_ist)

        # Construct command
        output_filename = "live_feed_last30sec.zip"

        command = [
            os.path.expanduser("~/.local/bin/iudx"),
            "--entity", "8a9c53c2-7291-454e-b173-2d9bfe893eb1",
            "--clientid", client_id,
            "--secret", secret,
            "--start", start_time,
            "--end", end_time,
            "--entity-type=resource",
            "--role=consumer",
            "--download=live_feed_last30sec",
            "--type=csv"
        ]

        ## Uncomment below line if you want to see full command
        # print("Running command:\n", " ".join(command))
        
        try:
            result = subprocess.run(command, capture_output=True, text=True, check=True)
            print(result.stdout)
            if result.stderr:
                print(result.stderr)

            # Move file to ./Data
            os.makedirs("Data", exist_ok=True)
            if os.path.exists(output_filename):
                shutil.move(output_filename, os.path.join("Data", output_filename))
                print(f"Saved to ./Data/{output_filename}")
            else:
                print(f"CSV file not found in current directory.")

        except subprocess.CalledProcessError as e:
            print("Error running command:")
            print(e.stderr or e.stdout)

# === Bind and Display ===
download_button.on_click(download_data)
display(client_id_widget, secret_widget, download_button, output)

Text(value='', description='Client ID:', placeholder='Enter your client ID')

Password(description='Secret:', placeholder='Enter your secret')

Button(button_style='success', description='Download Last 30s CSV', style=ButtonStyle())

Output()

### Visualizaion of bus location that traverse the route through selected stop location
For selected stop, 
- display positions of buses and stop in insteractive map
- display table with distance to the stop from these buses and ETA 

In [13]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from shapely.geometry import Point
from pyproj import Transformer
import zipfile
import ast

# === File Paths ===
stop_gpkg = "./Data/bus_stop_with_routes.gpkg"
route_gpkg = "./Data/bmtc_route_updated.gpkg"
zip_path = "./Data/live_feed_last30sec.zip"

# === Load Data ===
stops_gdf = gpd.read_file(stop_gpkg)  # assumed to be in EPSG:4326
routes_gdf = gpd.read_file(route_gpkg)  # assumed to be in EPSG:4326

# Read zipped CSV (live feed)
with zipfile.ZipFile(zip_path, 'r') as z:
    csv_file = [f for f in z.namelist() if f.endswith('.csv')][0]
    with z.open(csv_file) as f:
        obs_df = pd.read_csv(f)

# Extract lat/lon
obs_df[["long", "lat"]] = obs_df["location.coordinates"].apply(lambda x: pd.Series(ast.literal_eval(x)))

# Normalize route_id values: convert float-like strings to plain integer strings
obs_df["tripInfo.route_id"] = obs_df["tripInfo.route_id"].apply(
    lambda x: str(int(float(x))) if pd.notna(x) and str(x).replace('.', '', 1).isdigit() else None
)

# === Prepare dropdown ===
stop_display_map = {
    f"{row['stop_id']}_{row['stop_name']}": row['stop_id']
    for _, row in stops_gdf.iterrows()
}

stop_dropdown = widgets.Dropdown(
    options=sorted(stop_display_map.keys()),
    description="Select Stop:",
    layout=widgets.Layout(width='70%')
)

map_output = widgets.Output()
table_output = widgets.Output()

# === Scrollable Table Renderer ===
def display_scrollable_table(df, max_height="400px"):
    styles = f"""
    <style>
        .scroll-table-wrapper {{
            max-height: {max_height};
            overflow-y: auto;
            border: 1px solid #ccc;
        }}
        .scroll-table-wrapper table {{
            width: 100%;
            border-collapse: collapse;
        }}
        .scroll-table-wrapper th, .scroll-table-wrapper td {{
            border: 1px solid #ddd;
            padding: 4px 8px;
            text-align: left;
            font-size: 13px;
        }}
        .scroll-table-wrapper thead {{
            position: sticky;
            top: 0;
            background: #f9f9f9;
        }}
    </style>
    <div class="scroll-table-wrapper">{df.to_html(index=False)}</div>
    """
    display(HTML(styles))

# === Callback ===
def update_map(change):
    with map_output:
        clear_output(wait=True)
    with table_output:
        clear_output(wait=True)

    selected_label = change["new"]
    stop_id = stop_display_map[selected_label]
    stop_row = stops_gdf[stops_gdf["stop_id"] == int(stop_id)].iloc[0]

    stop_geom = stop_row.geometry  # EPSG:4326
    stop_coords = (stop_geom.y, stop_geom.x)

    try:
        # Normalize route_ids from stop file to string integers
        route_ids = [str(int(r)) for r in stop_row["new_route_ids"].split(",") if r.strip().isdigit()]
    except Exception:
        route_ids = []

    # Filter live feed for matching route_ids
    obs_filtered = obs_df[obs_df["tripInfo.route_id"].isin(route_ids)].copy()

    # Initialize map
    m = folium.Map(location=stop_coords, zoom_start=13)
    folium.Marker(
        location=stop_coords,
        popup=f"Stop: {stop_row['stop_name']} ({stop_id})",
        icon=folium.Icon(color='blue', icon='bus', prefix='fa')
    ).add_to(m)

    cluster = MarkerCluster().add_to(m)

    # Setup on-the-fly projection
    to_utm = Transformer.from_crs("EPSG:4326", "EPSG:7755", always_xy=True)

    # Project stop point for distance calculation
    stop_x, stop_y = to_utm.transform(stop_geom.x, stop_geom.y)
    stop_point_utm = Point(stop_x, stop_y)

    records = []

    for _, obs in obs_filtered.iterrows():
        route_id = obs["tripInfo.route_id"]
        vehicle_no = obs["license_plate"]
        obs_coords = (obs["lat"], obs["long"])

        folium.Marker(
            location=obs_coords,
            popup=f"Route: {route_id}<br>Vehicle: {vehicle_no}",
            icon=folium.Icon(color='red', icon='road', prefix='fa')
        ).add_to(cluster)

        # Project observation point
        obs_x, obs_y = to_utm.transform(obs["long"], obs["lat"])
        obs_point_utm = Point(obs_x, obs_y)

        # Get first matching route geometry and project for distance
        route_rows = routes_gdf[routes_gdf["new_route_id"].astype(str) == route_id]

        path_distance = None
        eta_min = None

        if not route_rows.empty:
            min_distance = float("inf")

            for _, row in route_rows.iterrows():
                route_geom = row.geometry

                if route_geom and not route_geom.is_empty and route_geom.length > 0:
                    # Project route to UTM
                    route_geom_proj = gpd.GeoSeries([route_geom], crs="EPSG:4326").to_crs(epsg=7755).iloc[0]

                    # Compute distances along the projected line
                    try:
                        dist_to_stop = route_geom_proj.project(stop_point_utm)
                        dist_to_obs = route_geom_proj.project(obs_point_utm)
                        dist = abs(dist_to_obs - dist_to_stop)
                    except Exception:
                        continue  # skip problematic geometry

                    if dist < min_distance:
                        min_distance = dist

            # Use min_distance if valid
            if min_distance < float("inf"):
                path_distance = min_distance / 1000  # convert meters to kilometers
                eta_min = round(path_distance / 0.5, 1)  # 30 km/h = 0.5 km/min


        records.append({
            "Route ID": route_id,
            "Vehicle No": vehicle_no,
            "Distance Along Route (km)": round(path_distance, 2) if path_distance is not None else "N/A",
            "ETA at 40 km/hr (min)": eta_min if eta_min is not None else "N/A"
        })

    with map_output:
        display(m)

    with table_output:
        df = pd.DataFrame(records)
        display_scrollable_table(df)

# === Show UI ===
stop_dropdown.observe(update_map, names="value")
display(stop_dropdown, map_output, table_output)


Dropdown(description='Select Stop:', layout=Layout(width='70%'), options=('20558_10th Cross Magadi Road', '205…

Output()

Output()